In [ ]:
!pip3 install -r requirements.txt


In [ ]:
!git clone https://github.com/justinphan3110/SciFive.git
!cp -r SciFive/biot5x/data .
!rm -r SciFive

In [ ]:
import torch
from tqdm import tqdm

from transformers import pipeline, AutoTokenizer, DataCollatorForSeq2Seq
from datasets import load_dataset, Dataset

from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead, set_seed
from trl.core import LengthSampler

In [ ]:
config = PPOConfig(
    model_name="justinphan3110/biot5_chemprot",
    # model_name="t5-base",
    learning_rate=1.41e-5,
    batch_size=256,
    forward_batch_size=256,
    eval_batch_size=32,
    input_length = 256,
    target_length = 5,
    metric = 'PRF1',
    ppo_epochs=1,
    init_kl_coef=0.0,
    log_with="tensorboard",
    remove_unused_columns=False,
    accelerator_kwargs={"logging_dir": "log"}
)
sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": config.forward_batch_size}

In [ ]:
# set seed before initializing value head for deterministic eval
set_seed(config.seed)

# Now let's build the model, the reference model, and the tokenizer.
model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(config.model_name)
ref_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)

In [ ]:
def preprocess_function(examples, input_length=config.input_length, target_length=config.target_length):
    model_inputs = tokenizer(
        examples["inputs"], max_length=input_length, truncation=True, padding=True
    )
    
    with tokenizer.as_target_tokenizer():
      labels = tokenizer(
          examples["labels"], max_length=target_length, truncation=True, padding=True
      )

    
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    return model_inputs

In [ ]:
input_column = "inputs"
target_column = "labels"
raw_datasets = {}
task = "chemprot"
id2label = {}
for line in open(f'data/{task}/label2id.tsv'):
    line = line.strip().split('\t')
    id_ = line[1]
    label = line[0]
    id2label[id_]=label

for file_ in ['train','test','dev']:
    with open(f"data/{task}/{file_}_blurb_text.tsv", "w") as out_file:
        with open(f"data/{task}/{file_}_blurb.tsv", "r") as file:
            for line in file:
                line = line.strip().split('\t')
                input_ = line[0]
                target = id2label[line[1]]
                out_file.write(f"{input_}\t{target}\n")
                


for file_ in ['train', 'dev', 'test']:
    inputs = []
    targets = []

    with open(f'data/{task}/{file_}_blurb_text.tsv', 'r', encoding='utf-8') as file:
        for i, line in enumerate(file):
            line = line.strip().split('\t')
            inputs.append(f'{line[0].strip()}')
            targets.append(f'{line[1].strip()}')
    
    
    dataset = Dataset.from_dict({input_column: inputs, target_column: targets})
    tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=1)
    tokenized_datasets.set_format(type="torch")
    raw_datasets[file_] = tokenized_datasets

In [ ]:
# We then build the PPOTrainer, passing the model, the reference model, the tokenizer
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")


## IMPORTANT: Need to be a multiple of batch size
train_datasets =  Dataset.from_dict(raw_datasets['train'][:512*20])

train_datasets.set_format(type="torch")

ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=train_datasets, data_collator=data_collator)

device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug
    print("device", device)
    
    
pipe = pipeline("text-classification")
classification_pipe = pipeline("text-classification", "justinphan3110/biolinkbert_chemprot", device=device)

In [ ]:
ppo_trainer.evaluate(raw_datasets['dev'])

In [ ]:
# We then define the arguments to pass to the `generate` function. These arguments
# are passed to the `generate` function of the PPOTrainer, which is a wrapper around
# the `generate` function of the trained model.
output_min_length = 2
output_max_length = config.target_length
output_length_sampler = LengthSampler(output_min_length, output_max_length)
generation_kwargs = { "max_length": config.target_length}
dataloader = torch.utils.data.DataLoader(train_datasets, collate_fn=data_collator, batch_size=config.forward_batch_size)


for epoch in range(5):
    out_dir = f"out/test_trl_biot5_{task}/checkpoint_{epoch}"
    for batch in tqdm(ppo_trainer.dataloader):
        query_tensors = batch["input_ids"]
        label_tensors = batch["labels"]

        outputs = ppo_trainer.generate(query_tensors, **generation_kwargs)
        response_tensors = list(outputs)
        texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        labels = tokenizer.batch_decode(label_tensors, skip_special_tokens=True)

        sent_kwargs['function_to_apply'] = 'sigmoid'
        sent_kwargs['return_all_scores'] = True
        pipe_outputs = classification_pipe(texts, **sent_kwargs)
        
        rewards = []
        for t,output, label in zip(texts, pipe_outputs, labels):
            if label == t:
                if label == '0':
                    reward = 1.0
                else:
                    reward = 1.0
            else:
                if label == '0':
                    reward = 0
                else: 
                    reward = 0
            rewards.append(torch.tensor(reward).to(device))
        
        assert len(rewards) == len(labels) == len(texts)
        #### Run PPO step
        stats = ppo_trainer.step(list(query_tensors), response_tensors, rewards)
        ppo_trainer.log_stats(stats, batch, rewards)
    print(f"saving pretrained epoch {epoch} to {out_dir}")
    ppo_trainer._save_pretrained(out_dir)
    ppo_trainer.evaluate(raw_dataset['dev'])